# Uptake Reactions
The notebook adds uptake reactions for [phenol](https://www.genome.jp/dbget-bin/www_bget?C00146), [vanillate](https://www.genome.jp/dbget-bin/www_bget?C06672), [4-hydroxybenzoate](https://www.genome.jp/dbget-bin/www_bget?C00156), [guaiacol](https://www.genome.jp/dbget-bin/www_bget?C01502), and [benzoate](https://www.genome.jp/dbget-bin/www_bget?C00180) to the *R.opacus* genome scale model.


###  Method
<ol>
<li>Add metabolites neccessary for aromatic growth</li>
<li>Add reactions to connect the metabolites to the metabolic network</li>
<li>Add phenol exchange, transport, and conversion to catechol</li>
</ol>

In [1]:
import cobra

### Load genome-scale model
Get *R.opacus* NCBI model generated by CarveMe

In [2]:
model = cobra.io.read_sbml_model("../GSMs/Ropacus_annotated_curated.xml")
model

Name,ropacus_annotated_curated
Memory address,0x07f4b679f51d0
Number of metabolites,1581
Number of reactions,2380
Number of groups,0
Objective expression,1.0*Growth - 1.0*Growth_reverse_699ae
Compartments,"cytosol, periplasm, extracellular space"


### Add neccessary metabolites to the model
Note: vanillate, 4-hydroxybenzoate, and benzoate are already in the model

In [3]:
# External phenol 
phenol_e = cobra.Metabolite(
    'phenol_e',
    formula='C6H6O',
    name='phenol',
    compartment='C_e')

# Cytosolic phenol
phenol_c = cobra.Metabolite(
    'phenol_c',
    formula='C6H6O',
    name='phenol',
    compartment='C_c')

# External vanillate
vanlt_e = cobra.Metabolite(
    'vanlt_e',
    formula='C8H7O4',
    name='Vanillate',
    compartment='C_e')

# Cytosolic vanillate = vanlt_c
# External 4-hydroxybenzoate = 4hbz_e
# Cytosolic 4-hydroxybenzoate = 4hbz_c

# External guaiacol
guaiacol_e = cobra.Metabolite(
    'guaiacol_e',
    formula='C7H8O2',
    name='guaiacol',
    compartment='C_e')

# Cytosolic guaiacol
guaiacol_c = cobra.Metabolite(
    'guaiacol_c',
    formula='C7H8O2',
    name='guaiacol',
    compartment='C_c')

# Cytosolic benzoate = bz_c
# External benzoate = bz_e

Add reactions for phenol exchange, transport, and [conversion to catechol](https://www.kegg.jp/entry/R10043) to the model

In [4]:
phenol_exchange = cobra.Reaction('EX_phenol_e')
phenol_exchange.name = 'Phenol exchange'
phenol_exchange.lower_bound = -1000.  
phenol_exchange.upper_bound = 1000.  
phenol_exchange.add_metabolites({
    phenol_e: -1.0,
})

phenol_transport = cobra.Reaction('phenol_transport')
phenol_transport.name = 'Phenol exchange'
phenol_transport.lower_bound = -1000. 
phenol_transport.upper_bound = 1000. 
phenol_transport.add_metabolites({
    phenol_e: -1.0,
    phenol_c:  1.0,
}) 

# EC 1.14.13.244
phenol_monooxygenase = cobra.Reaction('PHEMOX')
phenol_monooxygenase.name = 'Phenol exchange'
phenol_monooxygenase.lower_bound = 0.
phenol_monooxygenase.upper_bound = 1000.  # This is the default
phenol_monooxygenase.add_metabolites({
    phenol_c: -1.0,
    model.metabolites.get_by_id('nadh_c'):   -1.0,
    model.metabolites.get_by_id('h_c'): -1.0,
    model.metabolites.get_by_id('o2_c'): -1.0,
    model.metabolites.get_by_id('catechol_c'): 1.0,
    model.metabolites.get_by_id('nad_c'):  1.0,
    model.metabolites.get_by_id('h2o_c'):  1.0,
}) 


model.add_reactions([phenol_exchange])
model.add_reactions([phenol_transport])
model.add_reactions([phenol_monooxygenase])

Add reactions to exchange and transport vanillate

In [5]:
vanlt_exchange = cobra.Reaction('EX_vanlt_e')
vanlt_exchange.name = 'vanlt exchange'
vanlt_exchange.lower_bound = -1000.  
vanlt_exchange.upper_bound = 1000.  
vanlt_exchange.add_metabolites({
    vanlt_e: -1.0,
})

vanlt_transport = cobra.Reaction('vanlt_transport')
vanlt_transport.name = 'vanlt exchange'
vanlt_transport.lower_bound = -1000. 
vanlt_transport.upper_bound = 1000. 
vanlt_transport.add_metabolites({
    vanlt_e: -1.0,
    model.metabolites.get_by_id('vanlt_c'): 1.0
}) 

model.add_reactions([vanlt_exchange])
model.add_reactions([vanlt_transport])

No reactions need to be added for hydroxybenzoate consumption

Add reactions to exchange, transport, and [conversion to catechol](https://www.genome.jp/entry/R12301) guaiacol

In [6]:
guaiacol_exchange = cobra.Reaction('EX_guaiacol_e')
guaiacol_exchange.name = 'guaiacol exchange'
guaiacol_exchange.lower_bound = -1000.  
guaiacol_exchange.upper_bound = 1000.  
guaiacol_exchange.add_metabolites({
    guaiacol_e: -1.0,
})

guaiacol_transport = cobra.Reaction('guaiacol_transport')
guaiacol_transport.name = 'guaiacol exchange'
guaiacol_transport.lower_bound = -1000. 
guaiacol_transport.upper_bound = 1000. 
guaiacol_transport.add_metabolites({
    guaiacol_e: -1.0,
    guaiacol_c:  1.0,
}) 


# EC 1.14.14.-
guaiacol_o_demethylase = cobra.Reaction('GUADEM')
guaiacol_o_demethylase.name = 'Phenol exchange'
guaiacol_o_demethylase.lower_bound = 0.
guaiacol_o_demethylase.upper_bound = 1000.  # This is the default
guaiacol_o_demethylase.add_metabolites({
    guaiacol_c: -1.0,
    model.metabolites.get_by_id('nadph_c'):   -1.0,
    model.metabolites.get_by_id('o2_c'): -1.0,
    model.metabolites.get_by_id('catechol_c'): 1.0,
    model.metabolites.get_by_id('fald_c'):  1.0,
    model.metabolites.get_by_id('nadp_c'):  1.0,
    model.metabolites.get_by_id('h2o_c'):  1.0,
    
}) 

model.add_reactions([guaiacol_exchange])
model.add_reactions([guaiacol_transport])
model.add_reactions([guaiacol_o_demethylase])

No reactions need to be added for benzoate consumption

Confirm model can grow with phenol (medium composition proves no other carbon sources are consumed)

In [7]:
with model:
    medium = model.medium
    medium["EX_glc__D_e"] = 0.0
    medium["EX_phenol_e"] = 1.0
    medium["EX_vanlt_e"] = 0.0
    medium["EX_4hbz_e"] = 0.0
    medium["EX_guaiacol_e"] = 0.0
    medium["EX_bz_e"] = 0.0
    model.medium = medium
    [print(model.medium[m], m) for m in model.medium]
    solution = model.optimize()
    print(f'1 mmol of phenol can make {solution.objective_value:.3f} g of biomass')

10.0 EX_h2o_e
10.0 EX_h_e
10.0 EX_cl_e
10.0 EX_pi_e
10.0 EX_nh4_e
10.0 EX_fe3_e
10.0 EX_k_e
10.0 EX_ca2_e
10.0 EX_mg2_e
10.0 EX_mn2_e
10.0 EX_cobalt2_e
10.0 EX_zn2_e
10.0 EX_cu2_e
10.0 EX_o2_e
10.0 EX_fe2_e
10.0 EX_mobd_e
10.0 EX_so4_e
1.0 EX_phenol_e
1 mmol of phenol can make 0.084 g of biomass


Confirm model can grow with vanillate (medium composition proves no other carbon sources are consumed)

In [8]:
with model:
    medium = model.medium
    medium["EX_glc__D_e"] = 0.0
    medium["EX_phenol_e"] = 0.0
    medium["EX_vanlt_e"] = 1.0
    medium["EX_4hbz_e"] = 0.0
    medium["EX_guaiacol_e"] = 0.0
    medium["EX_bz_e"] = 0.0
    model.medium = medium
    [print(model.medium[m], m) for m in model.medium]
    solution = model.optimize()
    print(f'1 mmol of vanillate can make {solution.objective_value:.3f} g of biomass')

10.0 EX_h2o_e
10.0 EX_h_e
10.0 EX_cl_e
10.0 EX_pi_e
10.0 EX_nh4_e
10.0 EX_fe3_e
10.0 EX_k_e
10.0 EX_ca2_e
10.0 EX_mg2_e
10.0 EX_mn2_e
10.0 EX_cobalt2_e
10.0 EX_zn2_e
10.0 EX_cu2_e
10.0 EX_o2_e
10.0 EX_fe2_e
10.0 EX_mobd_e
10.0 EX_so4_e
1.0 EX_vanlt_e
1 mmol of vanillate can make 0.099 g of biomass


Confirm model can grow with 4-hydroxybenzoate (medium composition proves no other carbon sources are consumed)

In [9]:
with model:
    medium = model.medium
    medium["EX_glc__D_e"] = 0.0
    medium["EX_phenol_e"] = 0.0
    medium["EX_vanlt_e"] = 0.0
    medium["EX_4hbz_e"] = 1.0
    medium["EX_guaiacol_e"] = 0.0
    medium["EX_bz_e"] = 0.0
    model.medium = medium
    [print(model.medium[m], m) for m in model.medium]
    solution = model.optimize()
    print(f'1 mmol of 4-hydroxybenzoate can make {solution.objective_value:.3f} g of biomass')

10.0 EX_h2o_e
10.0 EX_h_e
1.0 EX_4hbz_e
10.0 EX_cl_e
10.0 EX_pi_e
10.0 EX_nh4_e
10.0 EX_fe3_e
10.0 EX_k_e
10.0 EX_ca2_e
10.0 EX_mg2_e
10.0 EX_mn2_e
10.0 EX_cobalt2_e
10.0 EX_zn2_e
10.0 EX_cu2_e
10.0 EX_o2_e
10.0 EX_fe2_e
10.0 EX_mobd_e
10.0 EX_so4_e
1 mmol of 4-hydroxybenzoate can make 0.080 g of biomass


Confirm model can grow with guaiacol (medium composition proves no other carbon sources are consumed)

In [10]:
with model:
    medium = model.medium
    medium["EX_glc__D_e"] = 0.0
    medium["EX_phenol_e"] = 0.0
    medium["EX_vanlt_e"] = 0.0
    medium["EX_4hbz_e"] = 0.0
    medium["EX_guaiacol_e"] = 1.0
    medium["EX_bz_e"] = 0.0
    model.medium = medium
    [print(model.medium[m], m) for m in model.medium]
    solution = model.optimize()
    print(f'1 mmol of guaiacol can make {solution.objective_value:.3f} g of biomass')

10.0 EX_h2o_e
10.0 EX_h_e
10.0 EX_cl_e
10.0 EX_pi_e
10.0 EX_nh4_e
10.0 EX_fe3_e
10.0 EX_k_e
10.0 EX_ca2_e
10.0 EX_mg2_e
10.0 EX_mn2_e
10.0 EX_cobalt2_e
10.0 EX_zn2_e
10.0 EX_cu2_e
10.0 EX_o2_e
10.0 EX_fe2_e
10.0 EX_mobd_e
10.0 EX_so4_e
1.0 EX_guaiacol_e
1 mmol of guaiacol can make 0.101 g of biomass


Confirm model can grow with benzoate (medium composition proves no other carbon sources are consumed)

In [11]:
with model:
    medium = model.medium
    medium["EX_glc__D_e"] = 0.0
    medium["EX_phenol_e"] = 0.0
    medium["EX_vanlt_e"] = 0.0
    medium["EX_4hbz_e"] = 0.0
    medium["EX_guaiacol_e"] = 0.0
    medium["EX_bz_e"] = 1.0
    model.medium = medium
    [print(model.medium[m], m) for m in model.medium]
    solution = model.optimize()
    print(f'1 mmol of benzoate can make {solution.objective_value:.3f} g of biomass')

10.0 EX_h2o_e
10.0 EX_h_e
10.0 EX_cl_e
10.0 EX_pi_e
10.0 EX_nh4_e
10.0 EX_fe3_e
10.0 EX_k_e
1.0 EX_bz_e
10.0 EX_ca2_e
10.0 EX_mg2_e
10.0 EX_mn2_e
10.0 EX_cobalt2_e
10.0 EX_zn2_e
10.0 EX_cu2_e
10.0 EX_o2_e
10.0 EX_fe2_e
10.0 EX_mobd_e
10.0 EX_so4_e
1 mmol of benzoate can make 0.094 g of biomass


In [12]:
cobra.io.write_sbml_model(model, "../GSMs/Ropacus_annotated_curated_with_aromatic_uptake.xml")
model

Name,ropacus_annotated_curated
Memory address,0x07f4b679f51d0
Number of metabolites,1586
Number of reactions,2388
Number of groups,0
Objective expression,1.0*Growth - 1.0*Growth_reverse_699ae
Compartments,"cytosol, periplasm, extracellular space"


# Test area (delete later)

function to print the flow of carbon source

In [ ]:
def carbon_follower(solution, metabolite):
    

phenol

In [35]:
with model:
    medium = model.medium
    medium["EX_glc__D_e"] = 0.0
    medium["EX_phenol_e"] = 1.0
    medium["EX_vanlt_e"] = 0.0
    medium["EX_4hbz_e"] = 0.0
    medium["EX_guaiacol_e"] = 0.0
    medium["EX_bz_e"] = 0.0
    model.medium = medium
    solution = model.optimize()
    print(f'1 mmol of phenol can make {solution.objective_value:.3f} g of biomass')

1 mmol of phenol can make 0.084 g of biomass


In [36]:
for r in model.metabolites.get_by_id('phenol_c').reactions:
    print(solution.fluxes[r.id], r)

1.0 phenol_transport: phenol_e <=> phenol_c
1.0 PHEMOX: h_c + nadh_c + o2_c + phenol_c --> catechol_c + h2o_c + nad_c


In [37]:
for r in model.metabolites.get_by_id('catechol_c').reactions:
    print(solution.fluxes[r.id], r)

0.0 BZDIOLDH: bz12diol_c + nad_c --> catechol_c + co2_c + nadh_c
0.0 GUADEM: guaiacol_c + nadph_c + o2_c --> catechol_c + fald_c + h2o_c + nadp_c
0.0 CATDOX: catechol_c + o2_c --> ccmuac_c + 2.0 h_c
1.0 PHEMOX: h_c + nadh_c + o2_c + phenol_c --> catechol_c + h2o_c + nad_c
1.0 CAT23DOX: catechol_c + o2_c --> 2hmcnsad_c


In [38]:
print(model.metabolites.get_by_id('2hmcnsad_c').name)
for r in model.metabolites.get_by_id('2hmcnsad_c').reactions:
    print(solution.fluxes[r.id], r)

2-Hydroxymuconate semialdehyde
1.0 HMSD: 2hmcnsad_c + h2o_c + nad_c --> 2hmc_c + 3.0 h_c + nadh_c
0.0 HMSH: 2hmcnsad_c + h2o_c --> for_c + 2.0 h_c + op4en_c
1.0 CAT23DOX: catechol_c + o2_c --> 2hmcnsad_c


In [39]:
print(model.metabolites.get_by_id('2hmc_c').name)
for r in model.metabolites.get_by_id('2hmc_c').reactions:
    print(solution.fluxes[r.id], r)

2-Hydroxymuconate
1.0 4OT: 2hmc_c <=> oxalc_c
1.0 HMSD: 2hmcnsad_c + h2o_c + nad_c --> 2hmc_c + 3.0 h_c + nadh_c


In [40]:
print(model.metabolites.get_by_id('oxalc_c').name)
for r in model.metabolites.get_by_id('oxalc_c').reactions:
    print(solution.fluxes[r.id], r)

4-oxalocrotonate
1.0 4OD: h_c + oxalc_c --> co2_c + op4en_c
1.0 4OT: 2hmc_c <=> oxalc_c


In [41]:
print(model.metabolites.get_by_id('op4en_c').name)
for r in model.metabolites.get_by_id('op4en_c').reactions:
    print(solution.fluxes[r.id], r)

2-Oxopent-4-enoate
0.0 HMSH: 2hmcnsad_c + h2o_c --> for_c + 2.0 h_c + op4en_c
1.0 OP4ENH: h2o_c + op4en_c --> 4h2opntn_c
0.0 HKNTDH: h2o_c + hkntd_c --> fum_c + h_c + op4en_c
1.0 4OD: h_c + oxalc_c --> co2_c + op4en_c
0.0 HKNDDH: h2o_c + hkndd_c --> h_c + op4en_c + succ_c
0.0 HMSH3: cchoxod_c + h2o_c --> ac_c + h_c + op4en_c


In [42]:
print(model.metabolites.get_by_id('4h2opntn_c').name)
for r in model.metabolites.get_by_id('4h2opntn_c').reactions:
    print(solution.fluxes[r.id], r)

4-Hydroxy-2-oxopentanoate
1.0 HOPNTAL: 4h2opntn_c --> acald_c + pyr_c
0.0 HOPNTAL3: 4h2opntn_c --> acald_c + pyr_c
1.0 OP4ENH: h2o_c + op4en_c --> 4h2opntn_c


guaiacol

In [43]:
with model:
    medium = model.medium
    medium["EX_glc__D_e"] = 0.0
    medium["EX_phenol_e"] = 0.0
    medium["EX_vanlt_e"] = 0.0
    medium["EX_4hbz_e"] = 0.0
    medium["EX_guaiacol_e"] = 1.0
    medium["EX_bz_e"] = 0.0
    model.medium = medium
    solution = model.optimize()
    print(f'1 mmol of guaiacol can make {solution.objective_value:.3f} g of biomass')

1 mmol of guaiacol can make 0.101 g of biomass


In [44]:
for r in model.metabolites.get_by_id('guaiacol_c').reactions:
    print(solution.fluxes[r.id], r)

1.0 GUADEM: guaiacol_c + nadph_c + o2_c --> catechol_c + fald_c + h2o_c + nadp_c
1.0 guaiacol_transport: guaiacol_e <=> guaiacol_c


In [45]:
for r in model.metabolites.get_by_id('catechol_c').reactions:
    print(solution.fluxes[r.id], r)

0.0 BZDIOLDH: bz12diol_c + nad_c --> catechol_c + co2_c + nadh_c
1.0 GUADEM: guaiacol_c + nadph_c + o2_c --> catechol_c + fald_c + h2o_c + nadp_c
0.0 CATDOX: catechol_c + o2_c --> ccmuac_c + 2.0 h_c
0.0 PHEMOX: h_c + nadh_c + o2_c + phenol_c --> catechol_c + h2o_c + nad_c
1.0 CAT23DOX: catechol_c + o2_c --> 2hmcnsad_c


Benzoate

In [20]:
with model:
    medium = model.medium
    medium["EX_glc__D_e"] = 0.0
    medium["EX_phenol_e"] = 0.0
    medium["EX_vanlt_e"] = 0.0
    medium["EX_4hbz_e"] = 0.0
    medium["EX_guaiacol_e"] = 0.0
    medium["EX_bz_e"] = 1.0
    model.medium = medium
    solution = model.optimize()
    print(f'1 mmol of guaiacol can make {solution.objective_value:.3f} g of biomass')

1 mmol of guaiacol can make 0.094 g of biomass


In [21]:
for r in model.metabolites.get_by_id('bz_c').reactions:
    print(solution.fluxes[r.id], r)

0.0 BCOALIG: atp_c + bz_c + coa_c + 4.0 h_c --> amp_c + benzcoa_c + ppi_c
0.9999908717167556 BZ12DOX: bz_c + h_c + nadh_c + o2_c --> bz12diol_c + nad_c
1.0 BZt: bz_e <=> bz_c
0.0 BZDH: bzal_c + h2o_c + nad_c --> bz_c + 2.0 h_c + nadh_c
9.128283244397295e-06 SUCBZT1: bz_c + 4.0 h_c + succoa_c <=> benzcoa_c + succ_c
0.0 BZt1pp: bz_p + h_p --> bz_c + h_c


In [48]:
print(model.metabolites.get_by_id('bz12diol_c').name)
for r in model.metabolites.get_by_id('bz12diol_c').reactions:
    print(solution.fluxes[r.id], r)

Cis-1,2-Dihydroxycyclohexa-3,5-diene-1-carboxylate
0.0 BZ12DOX: bz_c + h_c + nadh_c + o2_c --> bz12diol_c + nad_c
0.0 BZDIOLDH: bz12diol_c + nad_c --> catechol_c + co2_c + nadh_c


In [23]:
for r in model.metabolites.get_by_id('catechol_c').reactions:
    print(solution.fluxes[r.id], r)

0.9999908717167556 BZDIOLDH: bz12diol_c + nad_c --> catechol_c + co2_c + nadh_c
0.0 GUADEM: guaiacol_c + nadph_c + o2_c --> catechol_c + fald_c + h2o_c + nadp_c
0.0 CATDOX: catechol_c + o2_c --> ccmuac_c + 2.0 h_c
0.0 PHEMOX: h_c + nadh_c + o2_c + phenol_c --> catechol_c + h2o_c + nad_c
0.9999908717167556 CAT23DOX: catechol_c + o2_c --> 2hmcnsad_c


In [47]:
for r in model.metabolites.get_by_id('2hmcnsad_c').reactions:
    print(solution.fluxes[r.id], r)

1.0 HMSD: 2hmcnsad_c + h2o_c + nad_c --> 2hmc_c + 3.0 h_c + nadh_c
0.0 HMSH: 2hmcnsad_c + h2o_c --> for_c + 2.0 h_c + op4en_c
1.0 CAT23DOX: catechol_c + o2_c --> 2hmcnsad_c


4 Hydroxybenzoate

In [49]:
with model:
    medium = model.medium
    medium["EX_glc__D_e"] = 0.0
    medium["EX_phenol_e"] = 0.0
    medium["EX_vanlt_e"] = 0.0
    medium["EX_4hbz_e"] = 1.0
    medium["EX_guaiacol_e"] = 0.0
    medium["EX_bz_e"] = 0.0
    model.medium = medium
    solution = model.optimize()
    print(f'1 mmol of guaiacol can make {solution.objective_value:.3f} g of biomass')

1 mmol of guaiacol can make 0.080 g of biomass


In [50]:
print(model.metabolites.get_by_id('4hbz_c').name)
for r in model.metabolites.get_by_id('4hbz_c').reactions:
    print(solution.fluxes[r.id], r)

4-Hydroxybenzoate
0.0 BCOALIG2: 4hbz_c + atp_c + coa_c --> 4hbzcoa_c + amp_c + ppi_c
0.0024752761589086946 SUCBZT2: 4hbz_c + succoa_c <=> 4hbzcoa_c + succ_c
1.0 UHBZ1t_pp: 4hbz_p + h_p <=> 4hbz_c + h_c
0.0 VNDH_2: 4hbald_c + h2o_c + nad_c --> 4hbz_c + 2.0 h_c + nadh_c
0.0 HBZOPT: 4hbz_c + octdp_c --> 3ophb_c + ppi_c
0.9975247238410913 4HBHYOX: 4hbz_c + h_c + nadph_c + o2_c --> 34dhbz_c + h2o_c + nadp_c


In [51]:
print(model.metabolites.get_by_id('34dhbz_c').name)
for r in model.metabolites.get_by_id('34dhbz_c').reactions:
    print(solution.fluxes[r.id], r)

3,4-Dihydroxybenzoate
0.0 DHSKDH: 3dhsk_c --> 34dhbz_c + h2o_c
0.0 VNDH_3: 34dhbald_c + h2o_c + nad_c --> 34dhbz_c + 2.0 h_c + nadh_c
0.9975247238410913 PCADYOX: 34dhbz_c + o2_c --> CCbuttc_c + 2.0 h_c
0.0 VNTDM: h_c + nadh_c + o2_c + vanlt_c --> 34dhbz_c + fald_c + h2o_c + nad_c
0.0 3_4DHBZ1t_pp: 34dhbz_p + h_p <=> 34dhbz_c + h_c
0.9975247238410913 4HBHYOX: 4hbz_c + h_c + nadph_c + o2_c --> 34dhbz_c + h2o_c + nadp_c


In [52]:
print(model.metabolites.get_by_id('CCbuttc_c').name)
for r in model.metabolites.get_by_id('CCbuttc_c').reactions:
    print(solution.fluxes[r.id], r)

Cis,cis-Butadiene-1,2,4-tricarboxylate
0.9975247238410913 MUCCY_kt: CCbuttc_c + h_c --> 4cml_c
0.9975247238410913 PCADYOX: 34dhbz_c + o2_c --> CCbuttc_c + 2.0 h_c


In [53]:
print(model.metabolites.get_by_id('4cml_c').name)
for r in model.metabolites.get_by_id('4cml_c').reactions:
    print(solution.fluxes[r.id], r)

4-Carboxymuconolactone
0.0 CMLDC: 4cml_c + h_c --> 5odhf2a_c + co2_c
0.9975247238410913 4CMLCL_kt: 4cml_c + h_c --> 5odhf2a_c + co2_c
0.9975247238410913 MUCCY_kt: CCbuttc_c + h_c --> 4cml_c


In [54]:
print(model.metabolites.get_by_id('5odhf2a_c').name)
for r in model.metabolites.get_by_id('5odhf2a_c').reactions:
    print(solution.fluxes[r.id], r)

5-Oxo-4,5-dihydrofuran-2-acetate
0.0 CMLDC: 4cml_c + h_c --> 5odhf2a_c + co2_c
0.9975247238410913 4CMLCL_kt: 4cml_c + h_c --> 5odhf2a_c + co2_c
0.0 MUCLI: mucl_c <=> 5odhf2a_c
0.9975247238410913 OXOAEL: 5odhf2a_c + h2o_c --> 3oxoadp_c + h_c


In [55]:
print(model.metabolites.get_by_id('3oxoadp_c').name)
for r in model.metabolites.get_by_id('3oxoadp_c').reactions:
    print(solution.fluxes[r.id], r)

3-Oxoadipate
0.9975247238410913 3OADPCOAT: 3oxoadp_c + succoa_c --> oxadpcoa_c + succ_c
0.0 3OXOADPt_pp: 3oxoadp_p + h_p <=> 3oxoadp_c + h_c
0.9975247238410913 OXOAEL: 5odhf2a_c + h2o_c --> 3oxoadp_c + h_c


In [56]:
print(model.metabolites.get_by_id('oxadpcoa_c').name)
for r in model.metabolites.get_by_id('oxadpcoa_c').reactions:
    print(solution.fluxes[r.id], r)

3-Oxoadipyl-CoA
0.9975247238410913 3OADPCOAT: 3oxoadp_c + succoa_c --> oxadpcoa_c + succ_c
0.0 HADPCOADH3: 3hadpcoa_c + nad_c <=> h_c + nadh_c + oxadpcoa_c
0.9975247238410913 3OXCOAT: coa_c + oxadpcoa_c --> accoa_c + succoa_c


In [57]:
with model:
    medium = model.medium
    medium["EX_glc__D_e"] = 0.0
    medium["EX_phenol_e"] = 0.0
    medium["EX_vanlt_e"] = 1.0
    medium["EX_4hbz_e"] = 0.0
    medium["EX_guaiacol_e"] = 0.0
    medium["EX_bz_e"] = 0.0
    model.medium = medium
    solution = model.optimize()
    print(f'1 mmol of guaiacol can make {solution.objective_value:.3f} g of biomass')

1 mmol of guaiacol can make 0.099 g of biomass


In [58]:
print(model.metabolites.get_by_id('vanlt_c').name)
for r in model.metabolites.get_by_id('vanlt_c').reactions:
    print(solution.fluxes[r.id], r)

Vanillate
0.0 VANKpp: h_p + vanlt_p <=> h_c + vanlt_c
1.0 vanlt_transport: vanlt_e <=> vanlt_c
0.0 VNDH: h2o_c + nad_c + vanln_c --> 2.0 h_c + nadh_c + vanlt_c
1.0 VNTDM: h_c + nadh_c + o2_c + vanlt_c --> 34dhbz_c + fald_c + h2o_c + nad_c


In [59]:
print(model.metabolites.get_by_id('34dhbz_c').name)
for r in model.metabolites.get_by_id('34dhbz_c').reactions:
    print(solution.fluxes[r.id], r)

3,4-Dihydroxybenzoate
0.0 DHSKDH: 3dhsk_c --> 34dhbz_c + h2o_c
0.0 VNDH_3: 34dhbald_c + h2o_c + nad_c --> 34dhbz_c + 2.0 h_c + nadh_c
1.0 PCADYOX: 34dhbz_c + o2_c --> CCbuttc_c + 2.0 h_c
1.0 VNTDM: h_c + nadh_c + o2_c + vanlt_c --> 34dhbz_c + fald_c + h2o_c + nad_c
0.0 3_4DHBZ1t_pp: 34dhbz_p + h_p <=> 34dhbz_c + h_c
0.0 4HBHYOX: 4hbz_c + h_c + nadph_c + o2_c --> 34dhbz_c + h2o_c + nadp_c
